<a href="https://colab.research.google.com/github/sundargodina/game-recc/blob/main/game_recc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.cloud import bigquery
import pandas as pd

project_id = "famous-modem-466914-j8"

client = bigquery.Client(project=project_id)

# Helper function to load tables
def read_bq(table):
    return client.query(f"SELECT * FROM `{project_id}.games.{table}`").to_dataframe()

# Load tables
games_df = read_bq("cleaned_games")
users_df = read_bq("user_filtered_reviews")
recomm_df = read_bq("filtered_recommendations")
games_metadata_df = read_bq("metadata")


In [3]:
games_with_info = pd.merge(games_df, games_metadata_df, on='app_id')



In [42]:
games_with_info

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck,tags,description
0,1877360,Mars Rover Simulator,2022-02-22,True,False,False,Negative,9,11,9.99,9.99,0.0,True,"[Action, Automobile Sim, Space Sim, Action-Adv...",The landing was successful! The rover under yo...
1,316240,Professional Lumberjack 2015,2015-03-05,True,False,False,Negative,9,41,19.99,19.99,0.0,True,"[Simulation, Singleplayer]",Face the hard work of real lumberjacks as you ...
2,2203950,Magical Girl Celesphonia - Official Amane Cosp...,2023-03-02,True,False,False,Negative,15,33,0.00,0.00,0.0,True,"[Adventure, RPG, Indie]",
3,685370,Jewels of the Mysterious Woodland,2017-08-10,True,False,False,Mostly Negative,20,10,3.99,3.99,0.0,True,"[Match 3, Puzzle, Hidden Object, Family Friend...",Jewels of the Mysterious Woodland - The new ma...
4,1648580,Athletics 3: Summer Sports,2021-06-25,True,False,False,Mostly Negative,20,10,4.99,4.99,0.0,True,"[Sports, Simulation, eSports, Combat Racing, I...",Practice athletic sports in a realistic 3D env...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43400,1175222,talesshop puzzle 테일즈샵퍼즐 - 당신을 기다리는 여우 OST,2019-10-18,True,False,False,Positive,100,14,15.99,15.99,0.0,True,"[Indie, Casual, Free to Play]",
43401,1882820,My Little Dictator,2022-06-06,True,True,True,Positive,100,15,19.99,19.99,0.0,True,"[Visual Novel, Anime, World War II, Satire, Da...","A satirical dark-comedy parody of WW2, featuri..."
43402,1981860,Green Diver,2022-06-20,True,False,False,Positive,100,13,19.99,19.99,0.0,True,"[Adventure, Point & Click, Visual Novel, Choos...",This is a hidden object game in which you have...
43403,438710,Valentine's Day Set,2016-02-11,True,False,False,Positive,100,12,23.99,23.99,0.0,True,[Action],


In [ ]:


import requests
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import gc
from IPython.display import clear_output
import numpy as np

def is_empty_tags_safe(x):
    """Safe empty tag detection that handles arrays/lists properly"""
    try:
        # Handle NaN/None
        if pd.isna(x):
            return True

        # Handle arrays/lists
        if hasattr(x, '__len__'):
            if len(x) == 0:
                return True
            if len(x) == 1 and (str(x[0]).strip() == '' or pd.isna(x[0])):
                return True
            return False

        # Handle strings
        if isinstance(x, str):
            return x.strip() == '' or x == '[]'

        return False
    except:
        return True

# Colab-safe settings
MAX_WORKERS = 8  # Conservative for Colab
BATCH_SIZE = 2000  # Smaller batches to avoid memory issues
REQUEST_DELAY = 0.15  # Slightly slower to avoid IP blocks

request_lock = threading.Lock()
last_request_time = time.time()

def get_steam_tags_colab_safe(app_id: int) -> list:
    """Colab-safe Steam API call with conservative rate limiting"""
    global last_request_time

    with request_lock:
        current_time = time.time()
        time_since_last = current_time - last_request_time

        if time_since_last < REQUEST_DELAY:
            time.sleep(REQUEST_DELAY - time_since_last)

        last_request_time = time.time()

    try:
        url = f"https://store.steampowered.com/api/appdetails?appids={app_id}&l=english"
        response = requests.get(url, timeout=8)
        response.raise_for_status()

        data = response.json()

        if str(app_id) in data and data[str(app_id)]['success']:
            steam_data = data[str(app_id)]['data']

            tags = []

            # Get genres
            if 'genres' in steam_data:
                tags.extend([g['description'] for g in steam_data['genres']])

            # Get key categories
            if 'categories' in steam_data:
                useful_cats = {'Single-player', 'Multi-player', 'Co-op', 'Controller Support'}
                for cat in steam_data['categories']:
                    if cat['description'] in useful_cats:
                        tags.append(cat['description'])

            # Add developer
            if 'developers' in steam_data and steam_data['developers']:
                tags.append(steam_data['developers'][0])

            return list(set(tags)) if tags else ['Indie']

        return ['Unknown']

    except Exception as e:
        return None

def process_game_colab(args):
    """Process single game for Colab"""
    idx, app_id, title = args
    tags = get_steam_tags_colab_safe(app_id)
    return (idx, tags, title)

def colab_steam_tag_filler(df: pd.DataFrame, start_from: int = 0):
    """
    Colab-optimized tag filler with progress tracking and memory management
    """
    print("🔥 COLAB-OPTIMIZED STEAM TAG FETCHER")
    print("="*60)
    print(f"Settings: {MAX_WORKERS} workers, {BATCH_SIZE} batch size")

    # Find empty tags
    is_empty = df['tags'].apply(is_empty_tags_safe)
    empty_indices = df[is_empty].index.tolist()

    if start_from > 0:
        empty_indices = empty_indices[start_from:]
        print(f"📍 Resuming from position {start_from}")

    print(f"📊 Found {len(empty_indices)} games to process")
    print(f"⏱️ Estimated time: {(len(empty_indices) * REQUEST_DELAY / MAX_WORKERS / 60):.1f} minutes")

    df_result = df.copy()
    total_success = 0
    total_failed = 0

    # Process in memory-safe batches
    num_batches = (len(empty_indices) + BATCH_SIZE - 1) // BATCH_SIZE

    for batch_num in range(num_batches):
        start_idx = batch_num * BATCH_SIZE
        end_idx = min((batch_num + 1) * BATCH_SIZE, len(empty_indices))
        batch_indices = empty_indices[start_idx:end_idx]

        print(f"\n🚀 BATCH {batch_num + 1}/{num_batches}")
        print(f"   Processing games {start_from + start_idx + 1} to {start_from + end_idx}")

        # Prepare work for this batch
        work_items = [
            (idx, df.loc[idx, 'app_id'], df.loc[idx, 'title'][:30])
            for idx in batch_indices
        ]

        batch_success = 0
        batch_failed = 0

        # Process batch in parallel
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            future_to_idx = {executor.submit(process_game_colab, item): item for item in work_items}

            for i, future in enumerate(as_completed(future_to_idx), 1):
                idx, tags, title = future.result()

                if tags:
                    df_result.at[idx, 'tags'] = tags
                    batch_success += 1
                    total_success += 1
                else:
                    batch_failed += 1
                    total_failed += 1

                # Live progress update
                if i % 50 == 0 or i == len(batch_indices):
                    clear_output(wait=True)
                    print(f"🚀 BATCH {batch_num + 1}/{num_batches}")
                    print(f"   Progress: {i}/{len(batch_indices)} games")
                    print(f"   ✅ Success: {batch_success}, ❌ Failed: {batch_failed}")
                    print(f"   📈 Overall: {total_success} success, {total_failed} failed")

        # Save checkpoint after each batch
        checkpoint_name = f'colab_checkpoint_batch_{batch_num + 1}.csv'
        df_result.to_csv(checkpoint_name, index=False)

        print(f"💾 Checkpoint saved: {checkpoint_name}")
        print(f"⚡ Batch {batch_num + 1} completed!")

        # Memory cleanup
        gc.collect()

        # Brief pause between batches
        time.sleep(3)

    print(f"\n🎉 ALL BATCHES COMPLETED!")
    print(f"✅ Total successful: {total_success}")
    print(f"❌ Total failed: {total_failed}")
    print(f"📊 Success rate: {(total_success/(total_success+total_failed)*100):.1f}%")

    return df_result



updated_games = colab_steam_tag_filler(games_with_info)

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(games_with_info['description'])


In [51]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
tags_matrix = mlb.fit_transform(games_with_info['tags'])


In [52]:
from scipy.sparse import hstack

combined_matrix = hstack((tfidf_matrix, tags_matrix)).astype('float32')



In [53]:
import numpy as np
from scipy.sparse import hstack, csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import gc

# Filter out games with negative ratings
games_with_info = games_with_info[~games_with_info['rating'].isin(['Negative', 'Mostly Negative'])]

def compute_cosine_similarity_batched(matrix, batch_size=1000):
    # Convert to CSR format for efficient row slicing
    if not isinstance(matrix, csr_matrix):
        print("Converting matrix to CSR format...")
        matrix = matrix.tocsr()
        gc.collect()

    n_samples = matrix.shape[0]
    print(f"Processing {n_samples} samples with batch size {batch_size}")

    # Pre-allocate result matrix
    cosine_sim = np.zeros((n_samples, n_samples), dtype=np.float32)

    for i in range(0, n_samples, batch_size):
        end_i = min(i + batch_size, n_samples)
        batch_i = matrix[i:end_i]

        for j in range(0, n_samples, batch_size):
            end_j = min(j + batch_size, n_samples)
            batch_j = matrix[j:end_j]

            # Compute similarity for this batch pair
            sim_batch = cosine_similarity(batch_i, batch_j)
            cosine_sim[i:end_i, j:end_j] = sim_batch

            # Clear memory
            del sim_batch
            gc.collect()

        del batch_i
        gc.collect()

        print(f"Processed batch {i//batch_size + 1}/{(n_samples-1)//batch_size + 1}")

    return cosine_sim

# Usage
tfidf_matrix = tfidf.fit_transform(games_with_info['description'])
tags_matrix = mlb.fit_transform(games_with_info['tags'])
combined_matrix = hstack((tfidf_matrix, tags_matrix)).astype('float32')
cosine_sim = compute_cosine_similarity_batched(combined_matrix, batch_size=400)


Converting matrix to CSR format...
Processing 41681 samples with batch size 400
Processed batch 1/105
Processed batch 2/105
Processed batch 3/105
Processed batch 4/105
Processed batch 5/105
Processed batch 6/105
Processed batch 7/105
Processed batch 8/105
Processed batch 9/105
Processed batch 10/105
Processed batch 11/105
Processed batch 12/105
Processed batch 13/105
Processed batch 14/105
Processed batch 15/105
Processed batch 16/105
Processed batch 17/105
Processed batch 18/105
Processed batch 19/105
Processed batch 20/105
Processed batch 21/105
Processed batch 22/105
Processed batch 23/105
Processed batch 24/105
Processed batch 25/105
Processed batch 26/105
Processed batch 27/105
Processed batch 28/105
Processed batch 29/105
Processed batch 30/105
Processed batch 31/105
Processed batch 32/105
Processed batch 33/105
Processed batch 34/105
Processed batch 35/105
Processed batch 36/105
Processed batch 37/105
Processed batch 38/105
Processed batch 39/105
Processed batch 40/105
Processed

In [54]:
def get_recommendations(game_name, num_results, cosine_sim, dataframe):
    name = game_name.lower()
    query_game = dataframe[dataframe['title'].str.lower() == name]

    if query_game.empty:
        print(f"Game '{game_name}' not found.")
        return

    game_index = query_game.index[0]
    idx = dataframe.index.get_loc(game_index)

    scores = list(enumerate(cosine_sim[idx]))
    top_games = sorted(scores, key=lambda x: x[1], reverse=True)[1:num_results + 1]

    for i, _ in top_games:
        row = dataframe.iloc[i]
        print(f"\n🎮 {row['title']}")
        print(f"📜 Description: {row['description'][:200]}...")
        print(f"🏷️ Tags: {', '.join(row['tags'])}")
        print(f"⭐ Rating: {row['rating']}")
        print(f"💰 Price: ${row['price_final']}")


In [55]:
get_recommendations("Green Diver", 5, cosine_sim, games_with_info)


🎮 Adventures at the North Pole
📜 Description: Look for snow-covered tracks, collect evidence at the site of abandoned parking lots, communicate with Eskimo leaders and shamans, and find out what happened to the ship "Silent", its crew and captain...
🏷️ Tags: Casual, Adventure, Point & Click, Visual Novel, Clicker, 2D, Cute, Colorful, Atmospheric, Indie, Early Access, Story Rich, Singleplayer, Music
⭐ Rating: Positive
💰 Price: $9.99

🎮 Abedot Family Estate: Search For Hidden Objects
📜 Description: A game in the genre of "hidden object", in which you have to explore an old family mansion. Luxurious rooms, a library full of mysterious volumes, a winter garden and a creepy dungeon - explore locati...
🏷️ Tags: Casual, Adventure, Point & Click, Visual Novel, Interactive Fiction, Clicker, Cute, Colorful, Atmospheric, Drama, Indie, Story Rich, Singleplayer, Music, Hidden Object
⭐ Rating: Mostly Positive
💰 Price: $9.99

🎮 Moi Mei: Hidden Objects
📜 Description: A game in the genre of "hidden ob